# Quantitative Value Strategy

**Disclaimer:** _Investing in the stock market involves risk and can lead to monetary loss. This material is purely for educational purposes and should not be taken as professional investment advice. Invest at your own discretion._

Based on the course [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) by Nick McCullum from Youtube channel [freeCodeCamp.org](https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ).

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. 

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the '.csv' file is still in your working directory and import it with the following command:


In [2]:
stocks = pd.read_csv('./starter_files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio):

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)

In [5]:
print(data.status_code) # Should be 200

200


In [6]:
data.json()

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'EGES(LSA)GAODAQE CNTA/N BLMSL RTK',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'oafifilc',
 'close': None,
 'closeTime': None,
 'closeSource': 'iolaciff',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 139.326,
 'latestSource': 'Close',
 'latestTime': 'February 4, 2021',
 'latestUpdate': 1650531771670,
 'latestVolume': None,
 'iexRealtimePrice': 143.92,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1644966173882,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 139.1136,
 'previousVolume': 92136738,
 'change': 3.4464,
 'changePercent': 0.02606,
 'volume': None,
 'iexMarketPercent': 17.42815057345617,
 'iexVolume': 1121454,


## Parsing Our API Call

This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [7]:
data = data.json()

In [8]:
price = data['latestPrice']
price

139.326

In [9]:
pe_ratio = data['peRatio']
pe_ratio

37.47

## Executing A Batch API Call & Building Our DataFrame

Just like in the our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i+n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [12]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data frame one-by-one.

In [13]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

In [14]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.020,55.08,N/A
1,AAL,17.500,-1.27,N/A
2,AAP,165.780,23.66,N/A
3,AAPL,143.733,37.68,N/A
4,ABBV,111.480,24.3,N/A
...,...,...,...,...
500,YUM,105.360,30.69,N/A
501,ZBH,163.840,-224.28,N/A
502,ZBRA,417.960,48.6,N/A
503,ZION,50.050,21.32,N/A


## Removing Glamour Stocks

The opposite of "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50. 

In [15]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)

Remove Stocks with negative `Price-to-Earnings Ratio`

In [16]:
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]

In [17]:
final_dataframe = final_dataframe[:50]

In [18]:
final_dataframe.reset_index(inplace=True)

In [19]:
final_dataframe.drop('index',inplace=True, axis=1)

In [20]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,TEL,129.80,380.38,N/A
1,KMI,14.36,303.27,N/A
2,MAR,130.09,243.7,N/A
3,IQV,192.16,204.1,N/A
4,SBUX,107.60,183.93,N/A
5,CMG,1530.73,181.31,N/A
6,REG,51.00,180.48,N/A
7,NOW,604.97,175.4,N/A
8,DXCM,418.71,172.76,N/A
9,GLW,38.60,167.76,N/A


## Calculating the Number of Shares to Buy

We now need to calculate the number of shares we need to buy.

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [21]:
def portfolio_input():
    while True:
        portfolio_size = input('Enter the value of your portfolio:')

        try:
            val = float(portfolio_size)
            return val
            break
        except ValueError:
            print("That's not a number!\nPlease try again")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [22]:
portfolio_size = portfolio_input()

Enter the value of your portfolio:1000000


You can use now the global `portfolio_size` variable to calculate the number of shares that oru strategy should purchase.

In [23]:
position_size = portfolio_size/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] \
        = math.floor(position_size/final_dataframe.loc[i, 'Price'])

In [24]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,TEL,129.80,380.38,154
1,KMI,14.36,303.27,1392
2,MAR,130.09,243.7,153
3,IQV,192.16,204.1,104
4,SBUX,107.60,183.93,185
5,CMG,1530.73,181.31,13
6,REG,51.00,180.48,392
7,NOW,604.97,175.4,33
8,DXCM,418.71,172.76,47
9,GLW,38.60,167.76,518


## Building a Better (and More Realistic) Momentum Strategy

Every valuation metrics has certain flaws.

For example, the price-to-earnings ration doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using that price-to-book ratio.

Investors typically use a `composite` basket of validation metrics to build robust quantitative strategies. In this sector we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciacion and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided by the IEX Cloud API, and must be computed after pulling raw data. We'll start ... from scratch.

In [25]:
symbol = 'AAPL'
batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciacion and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit

(pe_ratio, pb_ratio, ps_ratio, ev_to_ebitda, ev_to_gross_profit)

(38.28, 35.92, 8.19, 27.23293479292905, 19.846488430812972)

Let's move on to building our DataFrame. You'll notice that I use abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [26]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [27]:
rv_dataframe = pd.DataFrame(columns=rv_columns)

In [28]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    np.NaN if ebitda is None or enterprise_value is None else enterprise_value / ebitda,
                    'N/A',
                    np.NaN if gross_profit is None or enterprise_value is None else enterprise_value / gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns
            ),
            ignore_index=True
        )

In [29]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.230,N/A,55.05,N/A,7.87,N/A,7.2,N/A,33.201512,N/A,13.298224,N/A,N/A
1,AAL,18.000,N/A,-1.3,N/A,-1.9,N/A,0.4477,N/A,-6.228818,N/A,1.449083,N/A,N/A
2,AAP,159.450,N/A,23.34,N/A,2.87,N/A,1.1,N/A,10.712041,N/A,2.399375,N/A,N/A
3,AAPL,142.891,N/A,38.65,N/A,36.4,N/A,7.85,N/A,26.703051,N/A,19.879047,N/A,N/A
4,ABBV,112.800,N/A,24.5,N/A,13.01,N/A,4.72,N/A,16.431324,N/A,9.514895,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.130,N/A,30.89,N/A,-4.06,N/A,5.67,N/A,22.818357,N/A,9.512261,N/A,N/A
501,ZBH,166.380,N/A,-228.62,N/A,2.83,N/A,4.93,N/A,26.990889,N/A,8.095745,N/A,N/A
502,ZBRA,417.500,N/A,47.39,N/A,11.61,N/A,5.1,N/A,32.261490,N/A,12.420631,N/A,N/A
503,ZION,48.920,N/A,21.02,N/A,1.11,N/A,2.62,N/A,11.484797,N/A,3.075520,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all the metrics we require are not available through the API we're using.

You can use pandas' `isnull` method to identify missing data:

In [30]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,210.10,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,244.04,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,35.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.07,N/A,14.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.78,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,159.50,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2140.25,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.21,N/A,32.16,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [31]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)

17

Dealing with missing data is an important topic in data science.

There are two main approaches:
* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column.

Here is the code to do this:

In [32]:
for column in ['Price-to-Earnings Ratio',
               'Price-to-Book Ratio',
               'Price-to-Sales Ratio',
               'EV/EBITDA',
               'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [33]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [34]:
metrics = {
    'Price-to-Earnings Ratio' :'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile'
}

In [35]:
from scipy.stats import percentileofscore as score

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [36]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.230,N/A,55.05,0.867327,7.87,0.732673,7.2000,0.80495,33.201512,0.847525,13.298224,0.788119,N/A
1,AAL,18.000,N/A,-1.30,0.151485,-1.90,0.049505,0.4477,0.0316832,-6.228818,0.029703,1.449083,0.0554455,N/A
2,AAP,159.450,N/A,23.34,0.471287,2.87,0.394059,1.1000,0.141584,10.712041,0.221782,2.399375,0.106931,N/A
3,AAPL,142.891,N/A,38.65,0.758416,36.40,0.962376,7.8500,0.827723,26.703051,0.776238,19.879047,0.934653,N/A
4,ABBV,112.800,N/A,24.50,0.491089,13.01,0.863366,4.7200,0.645545,16.431324,0.491089,9.514895,0.641584,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.130,N/A,30.89,0.637624,-4.06,0.0455446,5.6700,0.738614,22.818357,0.691089,9.512261,0.639604,N/A
501,ZBH,166.380,N/A,-228.62,0.0237624,2.83,0.392079,4.9300,0.693069,26.990889,0.780198,8.095745,0.526733,N/A
502,ZBRA,417.500,N/A,47.39,0.829703,11.61,0.835644,5.1000,0.706931,32.261490,0.843564,12.420631,0.762376,N/A
503,ZION,48.920,N/A,21.02,0.431683,1.11,0.110891,2.6200,0.407921,11.484797,0.265347,3.075520,0.150495,N/A


## Calculating the RV Score

We'll now calculate our RV score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing startegy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module:

In [37]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [38]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.230,N/A,55.05,0.867327,7.87,0.732673,7.2000,0.80495,33.201512,0.847525,13.298224,0.788119,0.808119
1,AAL,18.000,N/A,-1.30,0.151485,-1.90,0.049505,0.4477,0.0316832,-6.228818,0.029703,1.449083,0.0554455,0.0635644
2,AAP,159.450,N/A,23.34,0.471287,2.87,0.394059,1.1000,0.141584,10.712041,0.221782,2.399375,0.106931,0.267129
3,AAPL,142.891,N/A,38.65,0.758416,36.40,0.962376,7.8500,0.827723,26.703051,0.776238,19.879047,0.934653,0.851881
4,ABBV,112.800,N/A,24.50,0.491089,13.01,0.863366,4.7200,0.645545,16.431324,0.491089,9.514895,0.641584,0.626535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.130,N/A,30.89,0.637624,-4.06,0.0455446,5.6700,0.738614,22.818357,0.691089,9.512261,0.639604,0.550495
501,ZBH,166.380,N/A,-228.62,0.0237624,2.83,0.392079,4.9300,0.693069,26.990889,0.780198,8.095745,0.526733,0.483168
502,ZBRA,417.500,N/A,47.39,0.829703,11.61,0.835644,5.1000,0.706931,32.261490,0.843564,12.420631,0.762376,0.795644
503,ZION,48.920,N/A,21.02,0.431683,1.11,0.110891,2.6200,0.407921,11.484797,0.265347,3.075520,0.150495,0.273267


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [39]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace=True, drop=True)

In [40]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,43.24,N/A,-7.61,0.110891,0.5599,0.0594059,0.8074,0.0792079,4.603325,0.0435644,0.772045,0.0178218,0.0621782
1,AAL,18.00,N/A,-1.30,0.151485,-1.9000,0.049505,0.4477,0.0316832,-6.228818,0.029703,1.449083,0.0554455,0.0635644
2,UNM,25.73,N/A,5.45,0.19604,0.4845,0.0554455,0.4346,0.029703,2.682137,0.0356436,0.410993,0.0039604,0.0641584
3,MCK,194.11,N/A,-7.58,0.112871,-64.4400,0.00990099,0.1284,0.00594059,7.335299,0.0871287,2.720136,0.124752,0.0681188
4,F,11.77,N/A,-286.00,0.0217822,1.3600,0.167327,0.3533,0.0138614,4.597106,0.0415842,2.408476,0.108911,0.0706931
5,MET,53.60,N/A,8.76,0.205941,0.6589,0.0633663,0.7154,0.0633663,4.807997,0.0455446,0.707086,0.0118812,0.0780198
6,FTI,11.86,N/A,-0.90,0.156436,1.2400,0.135644,0.4016,0.0257426,3.482373,0.0376238,2.323957,0.10495,0.0920792
7,ALL,110.90,N/A,7.70,0.2,1.3400,0.161386,0.7527,0.0653465,2.449532,0.0336634,0.722753,0.0138614,0.0948515
8,HIG,53.03,N/A,10.78,0.229703,1.0400,0.0990099,0.8858,0.0891089,4.400850,0.039604,0.899717,0.0237624,0.0962376
9,BA,213.11,N/A,-10.21,0.10099,-6.9000,0.039604,2.1900,0.346535,-14.963133,0.0237624,-27.614521,0.0019802,0.102574


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy in each stock in our investment universe.

In [41]:
portfolio_size = portfolio_input()

Enter the value of your portfolio:250000


In [42]:
position_size = portfolio_size/len(rv_dataframe.index)

for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] \
        = math.floor(position_size/rv_dataframe.loc[i, 'Price'])

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [43]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,43.24,115,-7.61,0.110891,0.5599,0.0594059,0.8074,0.0792079,4.603325,0.0435644,0.772045,0.0178218,0.0621782
1,AAL,18.00,277,-1.30,0.151485,-1.9000,0.049505,0.4477,0.0316832,-6.228818,0.029703,1.449083,0.0554455,0.0635644
2,UNM,25.73,194,5.45,0.19604,0.4845,0.0554455,0.4346,0.029703,2.682137,0.0356436,0.410993,0.0039604,0.0641584
3,MCK,194.11,25,-7.58,0.112871,-64.4400,0.00990099,0.1284,0.00594059,7.335299,0.0871287,2.720136,0.124752,0.0681188
4,F,11.77,424,-286.00,0.0217822,1.3600,0.167327,0.3533,0.0138614,4.597106,0.0415842,2.408476,0.108911,0.0706931
5,MET,53.60,93,8.76,0.205941,0.6589,0.0633663,0.7154,0.0633663,4.807997,0.0455446,0.707086,0.0118812,0.0780198
6,FTI,11.86,421,-0.90,0.156436,1.2400,0.135644,0.4016,0.0257426,3.482373,0.0376238,2.323957,0.10495,0.0920792
7,ALL,110.90,45,7.70,0.2,1.3400,0.161386,0.7527,0.0653465,2.449532,0.0336634,0.722753,0.0138614,0.0948515
8,HIG,53.03,94,10.78,0.229703,1.0400,0.0990099,0.8858,0.0891089,4.400850,0.039604,0.899717,0.0237624,0.0962376
9,BA,213.11,23,-10.21,0.10099,-6.9000,0.039604,2.1900,0.346535,-14.963133,0.0237624,-27.614521,0.0019802,0.102574


## Formatting Our Excel Output

We will be using the XlsxWriter for Pyton to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [44]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, 'Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts and also symbols like % and $. We'll need four main formulas for our Excel document:

* String format for tickers
* `$XX.XX` format for stock prices
* `$XX.XXX` format for market capitalization
* Integer format for the numer of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built our formats in the last section of this course, I've included them below for you. Run this code before proceeding.

In [45]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00', 
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [46]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['Price-to-Earnings Ratio', float_format],
    'E': ['PE Percentile', percent_format],
    'F': ['Price-to-Book Ratio', float_format],
    'G': ['PB Percentile', percent_format],
    'H': ['Price-to-Sales Ratio', float_format],
    'I': ['PS Percentile', percent_format],
    'J': ['EV/EBITDA', float_format],
    'K': ['EV/EBITDA Percentile', percent_format],
    'L': ['EV/GP', float_format],
    'M': ['EV/GP Percentile', percent_format],
    'N': ['RV Score', float_format]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].write(f'{column}1',
                                          column_formats[column][0],
                                          column_formats[column][1])
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel file is very easy:

**THIS IS NOT AN INVESTMENT ADVICE!**

In [47]:
writer.save()